In [1]:
# different imports
import pandas as pd
import numpy as np
import os

In [2]:
# Read the sample file
co_file = os.path.join("client_start_folder","Co_600K_Jul2019_6M.pkl")
post_file = os.path.join("client_start_folder", "ONS", "data", "NSPL_AUG_2019_UK.pkl")
ds = pd.read_pickle(co_file)
ps = pd.read_pickle(post_file)

In [3]:
ds.shape

(600000, 131)

In [4]:
ps.shape

(2622934, 14)

In [5]:
from processing.DataProcessor import DataProcessor
data_proc = DataProcessor(co_file)

# find all columns that are complete i.e. have all fields filled in 
null_column_list = []

for c in ds.columns:
    if ds[c].isnull().sum() != 0:
        null_column_list.append(c)


zero_info_features = ["CompanyId", "CompanyNumber"]
only_one_value_features = ["Filled", "LimitedPartnershipsNumGenPartners", "LimitedPartnershipsNumLimPartners",\
                          "Status20190701"]
complicated_features = ["RegAddressAddressLine1", "RegAddressAddressLine2", "RegAddressCareOf", "RegAddressCounty", \
                        "RegAddressPOBox", "RegAddressPostCode", "RegAddressPostTown", "pcd", "oa11", "PreviousName_1CompanyName"]

to_remove_cols = list(set(zero_info_features+only_one_value_features+complicated_features+null_column_list))
#using set and the list to remove duplicates




In [6]:
#removing columsn we don't want to use
data_proc.remove_columns(to_remove_cols)

In [7]:
# changing columns to numerical/string type 
# before: to_num_cols = ["AccountsAccountRefDay", "AccountsAccountRefMonth", "oac1"]
to_num_cols = ["oac1"]
# before: to_str_cols = ["ru11ind"]
to_str_cols = []

X_train, X_test, y_train, y_test = data_proc.preprocess_data(to_num_cols, to_str_cols)
#the splitting of data etc is all done implicitly by data_proc.preprocess_data!!

In [8]:
#Training the Algorithm
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000, C=1e5) # parameters to be adjusted to make most optimal 

SKEvaluator should work with Linear Regression since RF class has implicit functions fit, predict, predict_proba which are used inside SKEvaluator

In [9]:
from ModelEvaluationTools.SKEvaluator import SKEvaluator
lr_model = SKEvaluator(classifier, "lr")

In [10]:
lr_model.fit(X_train, y_train)

In [11]:
test_auc = lr_model.evaluate(X_test, y_test)

Accuracy - Test: 0.9981833333333333
AUC - Test: 0.5882633649907423


In [12]:
# saves model to fitted_models folder
lr_model.save_model()